### 🏫 2. 실습 시나리오: "누가 어떤 강의를 샀을까?"

**상황 설정:**

  * 당신은 \*\*'파이썬 인강 사이트'\*\*의 데이터 분석가입니다.
  * 데이터가 DB 효율성을 위해 \*\*3개의 테이블(데이터프레임)\*\*로 쪼개져 저장되어 있습니다.
    1.  **학생 데이터 (`Students`)**: 학생 개인 정보
    2.  **강의 데이터 (`Courses`)**: 강의 상품 정보
    3.  **수강 신청 내역 (`Orders`)**: 누가, 언제, 무엇을 샀는지 기록 (여기에 Key가 다 모여있음\!)

**목표:** 흩어진 데이터를 **Key**를 기준으로 합쳐서 \*\*"가장 매출이 높은 강의 카테고리"\*\*를 분석하세요.

In [2]:
import pandas as pd

# 1. 학생 데이터 (Students) - 고유키: Student_ID
students = pd.DataFrame({
    'Student_ID': [101, 102, 103, 104, 105],  # PK (고유키): 학생을 식별
    'Name': ['철수', '영희', '민수', '지민', '성호'],
    'Region': ['Seoul', 'Busan', 'Seoul', 'Jeju', 'Incheon']
})

# 2. 강의 데이터 (Courses) - 고유키: Course_ID
courses = pd.DataFrame({
    'Course_ID': ['C001', 'C002', 'C003'],   # PK (고유키): 강의를 식별
    'Title': ['파이썬 기초', '머신러닝 실전', '딥러닝 심화'],
    'Category': ['Programming', 'AI', 'AI'],
    'Price': [50000, 120000, 150000]
})

# 3. 수강 신청 내역 (Orders) - 외래키: Student_ID, Course_ID
# 여기에는 구체적인 이름이나 강의명이 없고 'ID(숫자/코드)'만 있습니다. (DB 효율화)
orders = pd.DataFrame({
    'Order_ID': [1, 2, 3, 4, 5, 6], # 주문 번호
    'Student_ID': [101, 103, 101, 104, 102, 101], # FK (외래키): Students 테이블 참조
    'Course_ID': ['C001', 'C002', 'C003', 'C001', 'C002', 'C002'], # FK (외래키): Courses 테이블 참조
    'Order_Date': ['2024-03-01', '2024-03-02', '2024-03-03', '2024-03-04', '2024-03-05', '2024-03-06']
})

print("=== 1. 학생 데이터 (Students) ===")
print(students.head())
print("\n=== 2. 강의 데이터 (Courses) ===")
print(courses.head())
print("\n=== 3. 주문 내역 (Orders) - 여기가 핵심! ===")
print(orders.head())

=== 1. 학생 데이터 (Students) ===
   Student_ID Name   Region
0         101   철수    Seoul
1         102   영희    Busan
2         103   민수    Seoul
3         104   지민     Jeju
4         105   성호  Incheon

=== 2. 강의 데이터 (Courses) ===
  Course_ID    Title     Category   Price
0      C001   파이썬 기초  Programming   50000
1      C002  머신러닝 실전           AI  120000
2      C003   딥러닝 심화           AI  150000

=== 3. 주문 내역 (Orders) - 여기가 핵심! ===
   Order_ID  Student_ID Course_ID  Order_Date
0         1         101      C001  2024-03-01
1         2         103      C002  2024-03-02
2         3         101      C003  2024-03-03
3         4         104      C001  2024-03-04
4         5         102      C002  2024-03-05


In [4]:
merged_df = pd.merge(orders, students, on='Student_ID', how='left')
#print(merged_df)

full_data = pd.merge(merged_df, courses, on='Course_ID', how='left')

print("=== 🔗 모든 정보가 합쳐진 풀 데이터 ===")
print(full_data[['Order_ID', 'Name', 'Title', 'Price', 'Order_Date']])

=== 🔗 모든 정보가 합쳐진 풀 데이터 ===
   Order_ID Name    Title   Price  Order_Date
0         1   철수   파이썬 기초   50000  2024-03-01
1         2   민수  머신러닝 실전  120000  2024-03-02
2         3   철수   딥러닝 심화  150000  2024-03-03
3         4   지민   파이썬 기초   50000  2024-03-04
4         5   영희  머신러닝 실전  120000  2024-03-05
5         6   철수  머신러닝 실전  120000  2024-03-06


In [6]:
vip_user = full_data.groupby('Name')['Price'].sum().sort_values(ascending=False)
print("\n🏆 [VIP 학생 랭킹]")
print(vip_user)

category_revenue = full_data.groupby('Category')['Price'].sum()
print("\n💰 [카테고리별 매출]")
print(category_revenue)


🏆 [VIP 학생 랭킹]
Name
철수    320000
민수    120000
영희    120000
지민     50000
Name: Price, dtype: int64

💰 [카테고리별 매출]
Category
AI             510000
Programming    100000
Name: Price, dtype: int64


### 💡 핵심 요약

1.  **고유키(PK, Primary Key):** `Student_ID`, `Course_ID` 처럼 **중복되지 않고** 대상을 유일하게 식별하는 주민등록번호 같은 존재입니다.
2.  **외래키(FK, Foreign Key):** `Orders` 테이블에 있는 `Student_ID`처럼, **다른 테이블의 PK를 가리키는(참조하는)** 연결고리입니다.
3.  **왜 이렇게 나누나요?**
      * `Orders` 테이블에 학생 이름, 주소, 강의명, 가격을 매번 다 적으면 용량이 폭발하고 오타가 나서 관리가 안 됩니다.
      * 그래서 ID(코드)만 적어두고, 필요할 때만 `Merge(Join)`해서 쓰는 것이 \*\*관계형 데이터베이스(RDB)\*\*의 기본 원리입니다.

이 과정을 직접 수행해보시면, **"아, 데이터가 이렇게 연결되는구나\!"** 하고 SQL의 Join 개념까지 한 번에 잡으실 수 있습니다.

파일 생성해서 한번 실행해 보시겠어요? 😎